<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/BERTSumWithLemmatization%2C_StopWordRemeval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chatgpt.com/share/c64e87ae-aefb-4eef-af28-94d2025276d7

# Step 1: Install necessary dependencies

In [ ]:

!pip install transformers
!pip install torch
!pip install nltk
!pip install openpyxl

# Step 2: Import necessary libraries

In [1]:
# Step 2: Import necessary libraries
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
# Download NLTK data

In [2]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
file_path = './saved_file'
file_path = ('DataSampePilot.xlsx')

data = pd.read_excel(file_path)

In [ ]:
# Display the first few rows of the dataset
print("Original data:")
print(data.head())


# Step 3: Preprocess the text data

In [4]:
# Step 3: Preprocess the text data
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [5]:
def preprocess_text(text):
    # Tokenize into words
    words = word_tokenize(text)
    # Remove stopwords and perform lemmatization
    words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stop_words]
    # Join words back into a single string
    return ' '.join(words)

# Apply preprocessing to the Question_body and Answer_body columns
data['Question_body'] = data['Question_body'].apply(lambda x: preprocess_text(x) if pd.notnull(x) else "")
data['Answer_body'] = data['Answer_body'].apply(lambda x: preprocess_text(x) if pd.notnull(x) else "")


In [ ]:
# Verify preprocessing
print("Preprocessed data:")
print(data[['Question_body', 'Answer_body']].head())

# Step 4: Define the BertSum model for extractive summarization

In [7]:
# Step 4: Define the BertSum model for extractive summarization
class BertSum:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def summarize(self, text):
        sentences = sent_tokenize(text)
        if not sentences:
            return ""
        inputs = self.tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs)
        sentence_embeddings = outputs.last_hidden_state.mean(dim=1)
        scores = torch.norm(sentence_embeddings, dim=1)
        top_sentence_idxs = scores.topk(3).indices.tolist()
        summary = '. '.join([sentences[idx] for idx in top_sentence_idxs])
        return summary

In [8]:
# Initialize the model
bertsum = BertSum()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

# Step 5: Apply the model to the Question_body and Answer_body columns

In [9]:
# Step 5: Apply the model to the Question_body and Answer_body columns
def summarize_column(text, column_name):
    if pd.notnull(text):
        try:
            summary = bertsum.summarize(text)
            print(f"Original {column_name}: {text[:100]}...")  # Print first 100 characters
            print(f"Summary: {summary}\n")
            return summary
        except Exception as e:
            print(f"Error summarizing text: {e}")
            return ""
    return ""

data['Question_summary'] = data['Question_body'].apply(lambda x: summarize_column(x, 'Question_body'))
data['Answer_summary'] = data['Answer_body'].apply(lambda x: summarize_column(x, 'Answer_body'))


Original Question_body: Kinda new AWS . high-level question . Iâ€™m looking insight general architecture workflow , without ...
Summary: would React client update message ?. turn , server , code send queue message third party perform action .. waiting response , react app show status update , meaning different stage operation taking place server .

Original Question_body: spring boot microservices want use microservices client application ( front-end ) . use Spring MVC d...
Summary: spring boot microservices want use microservices client application ( front-end ) .. main logic application resides spring boot microservices .. use Spring MVC designing client side application , , client side application sends request microservices REST APIs use service , standard correct solution ?

Original Question_body: 'm trying properly design application according clean architecture , 'm struggling determine layer (...
Summary: also possible call UseCase repository .. RemoteData - load cache data AP

# Display the summarized data

In [10]:
# Display the summarized data
print("Summarized data:")
print(data[['Question_body', 'Question_summary', 'Answer_body', 'Answer_summary']].head())


Summarized data:
                                       Question_body  \
0  Kinda new AWS . high-level question . Iâ€™m lo...   
1  spring boot microservices want use microservic...   
2  'm trying properly design application accordin...   
3  heard .NET8 Microsoft gifted u totally & quot ...   
4  trying learn AWS service , mainly focused API ...   

                                    Question_summary  \
0  would React client update message ?. turn , se...   
1  spring boot microservices want use microservic...   
2  also possible call UseCase repository .. Remot...   
3  's trouble another time .. complication even i...   
4  Expose service REST endpoint Feature like auth...   

                                         Answer_body  \
0  send request , get response . order send respo...   
1  < blockquote > tl ; dr : Spring MVC contradict...   
2  Determining source information business logic ...   
3  always asked question : Microsoft template eve...   
4  Short answer : , n't proba

# Save the summarized data to a new Excel file

In [13]:
# Save the summarized data to a new Excel file
output_path = 'SummarizedData.xlsx'
data.to_excel(output_path, index=False, engine='openpyxl')



# Verify that the file has been saved correctly

In [ ]:

saved_data = pd.read_excel(output_path)
print("Saved summarized data:")
print(saved_data[['Question_summary', 'Answer_summary']].head())